In [ ]:
import os
import sys
import warnings

warnings.filterwarnings('ignore')

sys.path.append(os.path.join(os.getcwd(), 'workflow/lib'))
import ampseeker as amp

read_ann_field = amp.read_ann_field
vcf_to_excel = amp.vcf_to_excel
vcf_to_df = amp.vcf_to_df
split_rows_with_multiple_alleles = amp.split_rows_with_multiple_alleles
convert_genotype_to_alt_allele_count = amp.convert_genotype_to_alt_allele_count


In [ ]:
dataset = "nomads16"
wkdir = "../.."


# SNP data

In this notebook, we display the variant calling and annotation results as a pandas dataframe and save it as an excel spreadsheet for the user to explore or analyse further. If the DataFrame is too large to display, please use the .xlsx file. 

#### Target SNP data

In [ ]:
vcf_df = vcf_to_excel(
    vcf_path=f"{wkdir}/results/vcfs/targets/{dataset}.annot.vcf",
    excel_path=f"{wkdir}/results/vcfs/targets/{dataset}-snps.xlsx",
    convert_genotypes=True,
    split_multiallelic=True,
)
pd.set_option("display.max_rows", 1000, "display.max_columns", 1000)
vcf_df


---

In [ ]:
from IPython.display import display, Markdown
display(Markdown(f'<a href="{wkdir}/results/vcfs/targets/{dataset}-snps.xlsx">Target SNP data (.xlsx)</a>'))
display(Markdown(f'<a href="{wkdir}/results/vcfs/targets/{dataset}.annot.vcf">Target SNP data (.vcf)</a>'))


#### Whole-amplicon SNP data

For times sake, we do not write out whole-amplicon .xlsx by default - it is slow. 

In [ ]:
vcf_df = vcf_to_excel(
    vcf_path=f"{wkdir}/results/vcfs/amplicons/{dataset}.annot.vcf",
    excel_path=f"{wkdir}/results/vcfs/amplicons/{dataset}-snps.xlsx",
    convert_genotypes=True,
    split_multiallelic=True
)
vcf_df


---

In [ ]:
display(Markdown(f'<a href="{wkdir}/results/vcfs/amplicons/{dataset}-snps.xlsx">Whole amplicon SNP data (.xlsx)</a>'))
display(Markdown(f'<a href="{wkdir}/results/vcfs/amplicons/{dataset}.annot.vcf">Whole amplicon SNP data (.vcf)</a>'))
